# Lab 13: ML vs LLM - When to Use Which?

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab13_ml_vs_llm.ipynb)

Solve the same security problem with both ML and LLM, then compare results.

---

## 🌉 BRIDGE LAB: Transitioning from ML to LLM

> **Where you are in the curriculum:**
> - You've completed Labs 01-03 (ML track): classification, clustering, anomaly detection
> - You trained models on YOUR data with YOUR labels
> - Next up: Labs 04+ use Large Language Models (LLMs) via API calls
>
> **This lab bridges that gap** by showing you the same problem solved BOTH ways.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    YOUR LEARNING JOURNEY                             │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   LABS 01-03 (ML)              THIS LAB (03b)           LABS 04+    │
│   ─────────────────            ──────────────           ──────────  │
│                                                                     │
│   ┌─────────────┐                                    ┌───────────┐  │
│   │ Train your  │                                    │ Call API  │  │
│   │ own models  │ ──────────► COMPARISON ──────────► │ with your │  │
│   │ on data     │                 🌉                 │ prompts   │  │
│   └─────────────┘                                    └───────────┘  │
│                                                                     │
│   - scikit-learn          - When ML wins             - anthropic/   │
│   - .fit() / .predict()   - When LLM wins              openai SDK  │
│   - Feature engineering   - Hybrid patterns          - API keys    │
│   - Local models          - Cost/speed trade-offs    - Prompts     │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### Prerequisites
- **Required**: Lab 29 (classification concepts), Lab 32 (anomaly detection)
- **Helpful**: Lab 31 (prompt engineering basics)

### Why This Lab Matters

Many security teams struggle with this question: *"Should we use ML or LLM for this?"*

The answer is usually: **it depends** - and sometimes **both**.

This lab gives you hands-on experience with both approaches on the SAME problem, so you can make informed decisions in your own work.

---

## Learning Objectives
- Understand when to use ML vs LLM for security tasks
- Implement the same classifier with both approaches
- Compare speed, cost, accuracy, and flexibility
- Design hybrid systems that use both effectively

## The Challenge

You're a SOC analyst receiving thousands of log entries. Your task: classify each as **malicious** or **benign**.

We'll solve this with **both approaches** and compare!

**Next:** Lab 35 (LLM Log Analysis) - Start the LLM track!

In [ ]:
#@title Install dependencies (Colab only)
#@markdown Run this cell to install required packages in Colab

%pip install -q scikit-learn numpy pandas anthropic openai python-dotenv

In [ ]:
import numpy as np
import time
import os
from typing import List, Dict, Tuple
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("✅ Libraries loaded!")

## Understanding the Two Approaches

### Approach 1: Traditional ML

```
┌─────────────────────────────────────────────────────────────┐
│                    ML CLASSIFICATION                         │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   Log Entry                                                 │
│       │                                                     │
│       ▼                                                     │
│   Feature Extraction                                        │
│   ├── "failed" in text? → 1                                │
│   ├── "login" in text? → 1                                 │
│   ├── external IP? → 1                                     │
│   └── suspicious keywords? → 3                             │
│       │                                                     │
│       ▼                                                     │
│   [1, 1, 1, 3] → Model → 0.87 → MALICIOUS                  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### Approach 2: LLM Classification

```
┌─────────────────────────────────────────────────────────────┐
│                    LLM CLASSIFICATION                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   Log Entry                                                 │
│       │                                                     │
│       ▼                                                     │
│   Prompt: "You are a security analyst. Classify this log   │
│   entry as MALICIOUS or BENIGN. Explain your reasoning.    │
│                                                             │
│   Log: Failed login attempt for user admin from IP          │
│   185.143.223.47"                                           │
│       │                                                     │
│       ▼                                                     │
│   LLM Response:                                             │
│   "MALICIOUS - Multiple red flags:                         │
│    1. Failed login to privileged 'admin' account           │
│    2. External IP attempting internal access               │
│    3. Pattern consistent with brute force attack"          │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### Comparison at a Glance

| Factor | ML | LLM | Winner |
|--------|-----|-----|--------|
| **1,000 predictions** | <1 second | 5-30 minutes* | ML |
| **Cost for 10K logs** | ~$0 | ~$5-50* | ML |
| **Novel attack pattern** | May miss | Can reason | LLM |
| **Explanation for analyst** | "Feature X high" | Full context | LLM |
| **Works offline** | Yes | No (API) | ML |

*LLM times and costs vary by model, provider, and prompt length. Costs have dropped significantly - check current pricing.*

## Step 1: Create Sample Log Data

In [ ]:
# Sample log entries for classification
# In reality, these would come from your SIEM (Splunk, Elastic, etc.) or log aggregator

SAMPLE_LOGS = [
    # Malicious logs (label = 1)
    {"log": "Failed login attempt for user admin from IP 185.143.223.47", "label": 1},
    {"log": "PowerShell execution: -enc JABjAGwAaQBlAG4AdA...", "label": 1},
    {"log": "Multiple failed SSH attempts from 45.33.32.156", "label": 1},
    {"log": "Suspicious process: cmd.exe spawned by WINWORD.EXE", "label": 1},
    {"log": "Outbound connection to known C2 IP 91.219.28.103", "label": 1},
    {"log": "Registry modification: HKLM\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run", "label": 1},
    {"log": "Data exfiltration detected: 500MB uploaded to external IP", "label": 1},
    {"log": "Mimikatz signature detected in memory", "label": 1},
    {"log": "Failed login admin from 10.0.0.1 - brute force pattern", "label": 1},
    {"log": "Ransomware behavior: mass file encryption detected", "label": 1},
    # Benign logs (label = 0)
    {"log": "User john.doe logged in successfully from 192.168.1.50", "label": 0},
    {"log": "Scheduled task Windows Update ran successfully", "label": 0},
    {"log": "File backup completed: 1,234 files processed", "label": 0},
    {"log": "User password changed for account mary.smith", "label": 0},
    {"log": "Software update installed: Chrome 120.0.6099.109", "label": 0},
    {"log": "Print job completed for user finance_dept", "label": 0},
    {"log": "VPN connection established for remote_worker01", "label": 0},
    {"log": "Email sent from ceo@company.com to board@company.com", "label": 0},
    {"log": "Database backup completed successfully", "label": 0},
    {"log": "User logged out: session timeout after 30 minutes", "label": 0},
]

# Shuffle and split the data
import random
random.seed(42)
shuffled = SAMPLE_LOGS.copy()
random.shuffle(shuffled)

logs = [item["log"] for item in shuffled]
labels = [item["label"] for item in shuffled]

print(f"📊 Dataset: {len(logs)} log entries")
print(f"   Malicious: {sum(labels)}")
print(f"   Benign: {len(labels) - sum(labels)}")

## Step 2: ML Classifier

Build a traditional ML classifier using feature engineering.

In [ ]:
# ML Approach: Feature Engineering + Logistic Regression

# Define suspicious keywords that often appear in malicious logs
SUSPICIOUS_KEYWORDS = [
    "failed", "error", "unauthorized", "denied", "blocked",
    "malicious", "suspicious", "attack", "exploit", "inject",
    "powershell", "cmd.exe", "mimikatz", "ransomware", "c2",
    "encoded", "encrypted", "exfiltration", "brute"
]

def extract_features(log: str) -> List[float]:
    """
    Extract numerical features from a log entry.

    This is the key step in ML - converting text to numbers!

    Args:
        log: Raw log text

    Returns:
        List of feature values
    """
    log_lower = log.lower()

    features = [
        # Feature 1: Count of suspicious keywords
        sum(1 for kw in SUSPICIOUS_KEYWORDS if kw in log_lower),

        # Feature 2: Contains "failed" or "error"
        1 if "failed" in log_lower or "error" in log_lower else 0,

        # Feature 3: Contains IP address pattern
        1 if any(c.isdigit() and "." in log for c in log) else 0,

        # Feature 4: Log length (longer logs often more suspicious)
        len(log) / 100,  # Normalize

        # Feature 5: Contains "admin" or "root"
        1 if "admin" in log_lower or "root" in log_lower else 0,

        # Feature 6: Contains encoded/encrypted indicators
        1 if "enc" in log_lower or "base64" in log_lower else 0,
    ]

    return features

# Test feature extraction
print("Feature extraction examples:")
print(f"  Malicious log: {extract_features(SAMPLE_LOGS[0]['log'])}")
print(f"  Benign log: {extract_features(SAMPLE_LOGS[10]['log'])}")

In [ ]:
# Train the ML classifier

# Extract features for all logs
X = np.array([extract_features(log) for log in logs])
y = np.array(labels)

# Split data (use same indices for fair comparison later!)
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
    X, y, range(len(logs)), test_size=0.3, random_state=42
)

# Train model
start_time = time.time()
ml_model = LogisticRegression(random_state=42)
ml_model.fit(X_train, y_train)
ml_train_time = time.time() - start_time

# Make predictions
start_time = time.time()
ml_predictions = ml_model.predict(X_test)
ml_predict_time = time.time() - start_time

# Evaluate
ml_accuracy = accuracy_score(y_test, ml_predictions)
ml_precision = precision_score(y_test, ml_predictions, zero_division=0)
ml_recall = recall_score(y_test, ml_predictions, zero_division=0)

print("📊 ML CLASSIFIER RESULTS")
print("=" * 40)
print(f"Training time:    {ml_train_time*1000:.2f}ms")
print(f"Prediction time:  {ml_predict_time*1000:.2f}ms ({len(y_test)} samples)")
print(f"Accuracy:         {ml_accuracy:.1%}")
print(f"Precision:        {ml_precision:.1%}")
print(f"Recall:           {ml_recall:.1%}")
print(f"Cost:             $0.00")

## Step 3: LLM Classifier

Now let's use an LLM to classify the same logs.

> **Note**: This requires an API key. If you don't have one, you can still read the code to understand the approach.

In [ ]:
# LLM Classification approach
# This demonstrates the pattern - you'll need an API key to run

CLASSIFICATION_PROMPT = """You are a security analyst. Classify this log entry.

Log: {log_entry}

Respond with EXACTLY one word: MALICIOUS or BENIGN
"""

def llm_classify_simulated(log: str) -> str:
    """
    Simulated LLM classification for demo purposes.

    In production, this would call an actual LLM API.
    This simulation mimics LLM reasoning based on keywords.

    Args:
        log: Log entry to classify

    Returns:
        "MALICIOUS" or "BENIGN"
    """
    # Simulate LLM processing time (real API takes 500-2000ms)
    time.sleep(0.1)  # Reduced for demo

    # Simple rule-based simulation of LLM reasoning
    # Real LLM would understand context much better!
    malicious_indicators = [
        "failed", "attack", "malicious", "suspicious",
        "c2", "mimikatz", "ransomware", "exfiltration",
        "powershell -enc", "spawned by"
    ]

    log_lower = log.lower()
    if any(ind in log_lower for ind in malicious_indicators):
        return "MALICIOUS"
    return "BENIGN"

# Test on same test set
print("🤖 LLM CLASSIFIER (Simulated)")
print("=" * 40)

# Use same test indices for fair comparison!
test_logs = [logs[i] for i in test_idx]

start_time = time.time()
llm_predictions = []
for log in test_logs:
    pred = llm_classify_simulated(log)
    llm_predictions.append(1 if pred == "MALICIOUS" else 0)
llm_predict_time = time.time() - start_time

# Evaluate
llm_accuracy = accuracy_score(y_test, llm_predictions)
llm_precision = precision_score(y_test, llm_predictions, zero_division=0)
llm_recall = recall_score(y_test, llm_predictions, zero_division=0)

print(f"Prediction time:  {llm_predict_time*1000:.2f}ms ({len(y_test)} samples)")
print(f"Accuracy:         {llm_accuracy:.1%}")
print(f"Precision:        {llm_precision:.1%}")
print(f"Recall:           {llm_recall:.1%}")
print(f"Est. Cost:        ~${len(y_test) * 0.001:.2f} (at $0.001/call)")

## Step 4: The Hybrid Pattern

The best approach: **Use both!** ML handles bulk filtering, LLM handles uncertain cases.

```
┌─────────────────────────────────────────────────────────────┐
│                    HYBRID ARCHITECTURE                       │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   10,000 Log Entries                                        │
│           │                                                 │
│           ▼                                                 │
│   ┌───────────────────┐                                    │
│   │   ML FAST FILTER  │  ← Process ALL logs                │
│   │   (1 second)      │     Cost: ~$0                      │
│   └─────────┬─────────┘                                    │
│             │                                               │
│     ┌───────┴───────┐                                      │
│     │               │                                       │
│     ▼               ▼                                       │
│  BENIGN (9,500)  SUSPICIOUS (500)                          │
│  Auto-close      │                                          │
│                  ▼                                          │
│          ┌───────────────────┐                             │
│          │   LLM ANALYSIS    │  ← Only suspicious          │
│          │   (5 minutes)     │     Cost: ~$5               │
│          └─────────┬─────────┘                             │
│                    │                                        │
│            ┌───────┴───────┐                               │
│            │               │                                │
│            ▼               ▼                                │
│     False Positive    TRUE THREAT                           │
│     Auto-close        → Human Review                        │
│                                                             │
│   TOTAL: 10K logs in ~5 min, cost ~$5                      │
│   vs LLM-only: 10K logs in ~3 hours, cost ~$100            │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Hybrid approach: ML filter + LLM for uncertain cases

def hybrid_classify(log: str, model, threshold_low=0.3, threshold_high=0.7):
    """
    Hybrid classifier: ML for confident cases, LLM for uncertain ones.

    Args:
        log: Log entry to classify
        model: Trained ML model
        threshold_low: Below this = definitely benign
        threshold_high: Above this = definitely malicious

    Returns:
        Tuple of (prediction, used_llm)
    """
    # Step 1: Get ML probability
    features = np.array([extract_features(log)])
    probability = model.predict_proba(features)[0][1]  # P(malicious)

    # Step 2: Decide if confident enough
    if probability < threshold_low:
        return 0, False  # Definitely benign, ML only
    elif probability > threshold_high:
        return 1, False  # Definitely malicious, ML only
    else:
        # Uncertain - use LLM
        llm_result = llm_classify_simulated(log)
        return 1 if llm_result == "MALICIOUS" else 0, True

# Test hybrid approach
print("🔀 HYBRID CLASSIFIER")
print("=" * 40)

start_time = time.time()
hybrid_predictions = []
llm_calls = 0

for log in test_logs:
    pred, used_llm = hybrid_classify(log, ml_model)
    hybrid_predictions.append(pred)
    if used_llm:
        llm_calls += 1

hybrid_predict_time = time.time() - start_time

# Evaluate
hybrid_accuracy = accuracy_score(y_test, hybrid_predictions)
hybrid_precision = precision_score(y_test, hybrid_predictions, zero_division=0)
hybrid_recall = recall_score(y_test, hybrid_predictions, zero_division=0)

print(f"Prediction time:  {hybrid_predict_time*1000:.2f}ms")
print(f"LLM calls:        {llm_calls}/{len(y_test)} ({llm_calls/len(y_test)*100:.0f}%)")
print(f"Accuracy:         {hybrid_accuracy:.1%}")
print(f"Precision:        {hybrid_precision:.1%}")
print(f"Recall:           {hybrid_recall:.1%}")
print(f"Est. Cost:        ~${llm_calls * 0.001:.3f}")

## 🎉 Summary & Decision Framework

### Results Comparison

| Approach | Speed | Cost | Accuracy | Best For |
|----------|-------|------|----------|----------|
| **ML Only** | Fastest | Free | Good | High volume, known patterns |
| **LLM Only** | Slowest | Highest | Best | Novel patterns, explanations |
| **Hybrid** | Fast | Low | Great | Best of both worlds |

### When to Use What

```
START: What's your constraint?
│
├─► Speed/Volume critical (>100/sec)
│   └─► Use ML
│
├─► Cost critical (<$0.01/prediction)
│   └─► Use ML
│
├─► Need natural language explanation
│   └─► Use LLM (or Hybrid with LLM for uncertain)
│
├─► Handling novel/unknown patterns
│   └─► Use LLM (or Hybrid)
│
├─► Must work offline/air-gapped
│   └─► Use ML
│
└─► Want best of both worlds
    └─► Use Hybrid (ML filter → LLM verify)
```

## Key Takeaways

1. **ML excels at** high-volume, known-pattern, low-cost scenarios
2. **LLM excels at** reasoning, flexibility, and explanation
3. **Hybrid is often best** - ML handles bulk, LLM handles edge cases
4. **Know your constraints** - speed, cost, accuracy, explainability
5. **Measure both** - don't assume, test on your data

## Next Steps

- **Lab 35**: Deep dive into LLM prompt engineering
- **Lab 36**: Build agents that combine ML + LLM
- **Lab 23**: Production hybrid detection pipeline